# Libraries

In [1]:
!pip install --upgrade certifi
!pip install --upgrade pip
!pip install --trusted-host pypi.org pooch
!pip install --index-url=http://pypi.python.org/simple/ --trusted-host pypi.python.org pooch
!pip install --upgrade matplotlib
!pip install docx2pdf
!pip install seaborn
!pip install folium
!pip install openpyxl
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.backends.backend_agg as agg
from io import BytesIO
from scipy import stats
from scipy.stats import genextreme as gev
import pooch
import tempfile
import docx2pdf
import folium
from folium.plugins import MarkerCluster

Looking in indexes: http://pypi.python.org/simple/


/home/user3/miniconda3/envs/ankit_intern/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Class

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
from scipy.stats import genextreme as gev
from scipy.optimize import minimize
from concurrent.futures import ProcessPoolExecutor

class AnnualMaxPrecipitationAnalyzer:
    def __init__(self, folder_path, excel_file_path, intersection_point_path, GMT_File_Path,data_start_year = 1951 ,start_year=1951, end_year=2022,return_periods=[25, 50, 100]):
        # initialize inputs
        self.folder_path = folder_path
        self.excel_file_path = excel_file_path
        self.intersection_point_path = intersection_point_path
        self.GMT_File_Path = GMT_File_Path
        self.start_year = start_year
        self.end_year = end_year
        self.data_start_year = data_start_year
        self.return_periods = return_periods
        #read file
        self.df = pd.read_excel(self.excel_file_path)
        self.intersection_df = pd.read_excel(self.intersection_point_path)
        self.annual_precipitation_latitude_longitude = {}
        self.colors = sns.color_palette("husl", len(os.listdir(folder_path)))
        self.years = list(range(self.start_year, self.end_year + 1))
        # initialize variables
        self.risk_factors = {}
        self.shape = None
        self.loc = None
        self.scale = None
        self.resultsNon_Stationary_Stationary_Analysis_location_vary = []
        self.resultsNon_Stationary_Stationary_Analysis_splitwise = []
        self.daily_precipitation = []



    def process_files(self, duration=1):
        self.years_range = pd.date_range(start=f'{self.start_year}-01-01', end=f'{self.end_year}-12-31', freq='Y').year
        for i, filename in enumerate(os.listdir(self.folder_path)):
            if filename.startswith("data_"):
                lat_lon = filename.replace("data_", "").split("_")
                latitude, longitude = float(lat_lon[0]), float(lat_lon[1])
                if ((self.df['Latitude'] == latitude) & (self.df['Longitude'] == longitude)).any():
                    file_path = os.path.join(self.folder_path, filename)
                    with open(file_path, 'r') as file:
                        lines = file.readlines()
                    data_list = [line.strip().split() for line in lines]
                    columns = ['Precipitation(mm)', 'Tmax(Degree Celcius)', 'Tmin(Degree Celcius)', 'Wind(m/s)']
                    df_file = pd.DataFrame(data_list, columns=columns)
                    df_file['Date'] = pd.date_range(start=f'{self.data_start_year}-01-01', periods=len(df_file), freq='D')
                    start_date = f'{self.end_year + 1}-01-01'
                    df_filtered = df_file[(df_file['Date'] < start_date)].copy()
                    df_filtered['Precipitation(mm)'] = pd.to_numeric(df_filtered['Precipitation(mm)'], errors='coerce')
                    self.daily_precipitation.append(df_filtered[['Date', 'Precipitation(mm)']])

        # Combine daily precipitation data from all files
        Daily_Precipitation_All_Points_In_Catchment = pd.concat(self.daily_precipitation)
        Daily_Precipitation_Catchment = Daily_Precipitation_All_Points_In_Catchment.groupby('Date')['Precipitation(mm)'].mean().reset_index()

        # Calculate annual maximum precipitation for the specified duration
        Daily_Precipitation_Catchment['Year'] = Daily_Precipitation_Catchment['Date'].dt.year
        Annual_Maxima_Precipitation_Catchment = []

        for year in self.years_range:
            year_data = Daily_Precipitation_Catchment[Daily_Precipitation_Catchment['Year'] == year]
            if len(year_data) > 0:
                rolling_sum = year_data['Precipitation(mm)'].rolling(window=duration, min_periods=1).sum()
                max_precip = rolling_sum.max()
                Annual_Maxima_Precipitation_Catchment.append(max_precip if np.isfinite(max_precip) else np.nan)
            else:
                Annual_Maxima_Precipitation_Catchment.append(np.nan)

        # Now Annual_Maxima_Precipitation_Catchment will have the same length as self.years_range
        self.Annual_Maxima_Precipitation_Catchment = Annual_Maxima_Precipitation_Catchment
#         print(len(self.Annual_Maxima_Precipitation_Catchment))

        # Read the GMT file correctly
        GMT_File = pd.read_csv(self.GMT_File_Path, delim_whitespace=True, header=None, skiprows=1)
        columns = ['Years', 'Historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
        GMT_File.columns = columns
        
        # Extract the years and Global Mean Temperature columns
        GMT = GMT_File[['Years', 'Historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']]
        GMT[['Historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']] -= 273.15
        
        # Filter data to include only years present in self.years
        GMT_start_to_end_year = GMT[GMT['Years'].isin(self.years)]
        GMT_start_to_end_year = GMT_start_to_end_year.reset_index(drop=True)
        
        # Create a DataFrame to combine average annual precipitation and global mean temperature
        Analyze_Data = pd.DataFrame({
            'Years': self.years,
            'Average Annual Precipitation': self.Annual_Maxima_Precipitation_Catchment
        })
        
        # Merge the dataframes on 'Years'
        self.Analyze_Data = pd.merge(Analyze_Data, GMT_start_to_end_year, on='Years', how='inner')
    
        # print(self.Analyze_Data)
    def Non_Stationary_Stationary_Analysis_Using_Window_Size(self,window_size=1):
       
        # Perform Stationary GEV Analysis to get initial parameters
        shape_SS, loc_SS, scale_SS = gev.fit(self.Annual_Maxima_Precipitation_Catchment)
        print(f"Stationary GEV Parameters: loc={loc_SS}, scale={scale_SS}, shape={shape_SS}")
   
        # Calculate return levels using stationary parameters
        stationary_return_levels = {}
        for rp in self.return_periods:
            stationary_return_levels[rp] = gev.ppf(1 - 1/rp, c=shape_SS, loc=loc_SS, scale=scale_SS)
   
        gev_parameters_list = []  # Initialize a list to collect parameters
        return_levels = {rp: [] for rp in self.return_periods}  # Dictionary to store return levels for each return period

        for i in range(len(self.Annual_Maxima_Precipitation_Catchment) - window_size + 1):
            window_years = self.years_range[i:i + window_size]
            window_precip = self.Annual_Maxima_Precipitation_Catchment[i:i + window_size]

            # Remove non-finite values from the window_precip
            window_precip = [x for x in window_precip if np.isfinite(x)]

            if len(window_precip) > 0:  # Only fit if there are finite values
                shape, loc, scale = gev.fit(window_precip)
                gev_parameters_list.append({'Year': int(window_years[-1]), 'Shape': shape, 'Location': loc, 'Scale': scale})

                # Calculate return levels for each return period
                for rp in self.return_periods:
                    return_level = gev.ppf(1 - 1/rp, c=shape, loc=loc, scale=scale)
                    return_levels[rp].append(return_level)
                   
       
        # Convert the list of dictionaries to a DataFrame and append to self.gev_parameters
        self.gev_parameters = pd.DataFrame(gev_parameters_list)
        mean_location = self.gev_parameters['Location'].mean()
        mean_scale = self.gev_parameters['Scale'].mean()
        mean_shape = self.gev_parameters['Shape'].mean()
       
        # Calculate return levels for each return period for last year
        non_stationary_return_levels_mean_paramters = {}
        for rp in self.return_periods:
            non_stationary_return_levels_mean_paramters[rp] = gev.ppf(1 - 1/rp, c=mean_shape, loc=mean_location, scale=mean_scale)

        # Check if catchment ID is present in the intersection DataFrame
        filename = os.path.splitext(os.path.basename(self.excel_file_path))[0]
        self.catchment_id = int(filename)
        self.intersection_df['Catchment ID'] = self.intersection_df['Catchment ID'].astype(int)
        filtered_df = self.intersection_df[self.intersection_df['Catchment ID'] == self.catchment_id]
        if not filtered_df.empty:
            self.Latitude = filtered_df['Latitude'].values[0]
            self.Longitude = filtered_df['Longitude'].values[0]
            self.resultsNon_Stationary_Stationary_Analysis_splitwise.append({
            'Catchment ID': self.catchment_id,
                'Latitude' : self.Latitude,
                'Longitude' : self.Longitude,
            'Stationary Location parameter': loc_SS,
            'Stationary Shape parameter': shape_SS,
            'Stationary Scale parameter': scale_SS,
            **{f'Stationary Return value for {rp}-year': stationary_return_levels[rp].item() if isinstance(stationary_return_levels[rp], np.ndarray) else stationary_return_levels[rp] for rp in self.return_periods},
            'Location parameter Mean': mean_location,
            'Shape parameter Mean': mean_shape,
            'Scale parameter Mean': mean_scale,        
            **{f'Non-Stationary Return value for {rp}-year': non_stationary_return_levels_mean_paramters[rp].item() if isinstance(non_stationary_return_levels_mean_paramters[rp], np.ndarray) else non_stationary_return_levels_mean_paramters[rp] for rp in self.return_periods},
            **{f'Change in Precipitation for return period {rp}-year': (non_stationary_return_levels_mean_paramters[rp] - stationary_return_levels[rp]).item() if isinstance((non_stationary_return_levels_mean_paramters[rp] - stationary_return_levels[rp]), np.ndarray) else non_stationary_return_levels_mean_paramters[rp] - stationary_return_levels[rp] for rp in self.return_periods}  
            })
        else:
            self.Latitude = None
            self.Longitude = None
            print(f"No matching catchment ID {self.catchment_id} found in the intersection DataFrame. Skipping.")

        print(self.catchment_id)
        plt.figure(figsize=(15, 5))

        plt.subplot(1, 3, 1)
        plt.plot(self.gev_parameters['Year'], self.gev_parameters['Location'], marker='o')
        plt.title('Location Parameter Over Time')
        plt.xlabel('Year')
        plt.ylabel('Location Parameter')

        plt.subplot(1, 3, 2)
        plt.plot(self.gev_parameters['Year'], self.gev_parameters['Scale'], marker='o')
        plt.title('Scale Parameter Over Time')
        plt.xlabel('Year')
        plt.ylabel('Scale Parameter')

        plt.subplot(1, 3, 3)
        plt.plot(self.gev_parameters['Year'], self.gev_parameters['Shape'], marker='o')
        plt.title('Shape Parameter Over Time')
        plt.xlabel('Year')
        plt.ylabel('Shape Parameter')

        plt.tight_layout()
        plt.show()

        # Plot return levels for each return period
        plt.figure(figsize=(15, 5))
        for rp in self.return_periods:
            plt.plot(self.gev_parameters['Year'], return_levels[rp], marker='o', label=f'{rp}-Year Return Level')

        plt.title('Return Levels Over Time')
        plt.xlabel('Year')
        plt.ylabel('Return Level')
        plt.legend()
        plt.tight_layout()
        plt.show()
       
       
    def Non_Stationary_Stationary_Analysis_location_vary(self,type_model):
        Dataset = self.Analyze_Data
        years = Dataset['Years'].values
        annual_max_precip = Dataset['Average Annual Precipitation'].values
        gmt = Dataset[type_model].values
        # print(gmt)
        gmt_end = Dataset[Dataset['Years']==self.end_year][type_model].values
        # Ensure that the data does not contain any NaN values
        mask = ~np.isnan(annual_max_precip)
        years = years[mask]
        annual_max_precip = annual_max_precip[mask]
        gmt = gmt[mask]
   
        # Perform Stationary GEV Analysis to get initial parameters
        shape, loc, scale = gev.fit(annual_max_precip)
        # print(f"Stationary GEV Parameters: loc={loc}, scale={scale}, shape={shape}")
   
        # Calculate return levels using stationary parameters
        stationary_return_levels = {}
        for rp in self.return_periods:
            stationary_return_levels[rp] = gev.ppf(1 - 1/rp, c=shape, loc=loc, scale=scale)
   
        # Use stationary parameters as initial guess
        initial_params = [loc, 0, scale, shape]

        # Negative Log-Likelihood Function
        def negative_log_likelihood(params, x, t, g):
            beta0, beta1, beta2, sigma, xi = params
            mu = beta0 + beta1 * (t - 1951) + beta2 * g  # Location depends on years and GMT
            if sigma <= 0:
                return np.inf
            return -np.sum(gev.logpdf(x, c=xi, loc=mu, scale=sigma))

        # Optimization
        initial_params = [loc, 0, 0, scale, shape]
        result = minimize(negative_log_likelihood, initial_params, args=(annual_max_precip, years, gmt),
                         method='SLSQP', bounds=[(-np.inf, np.inf), (-np.inf, np.inf), (-np.inf, np.inf), (0, np.inf), (-1, 1)])
        beta0, beta1, beta2, sigma, xi = result.x

        # Calculate non-stationary return levels
        non_stationary_return_levels = {}
        for rp in self.return_periods:
            non_stationary_return_levels[rp] = gev.ppf(1 - 1/rp, c=xi, loc=beta0 + beta1 * (years - self.start_year) + beta2 * gmt, scale=sigma)
        # Calculate return levels for each return period for last year
        non_stationary_return_levels_end_year = {}
        for rp in self.return_periods:
            non_stationary_return_levels_end_year[rp] = gev.ppf(1 - 1/rp, c=xi, loc=beta0 + beta1 * (self.end_year - self.start_year) + beta2 * gmt_end, scale=sigma)
        
        # Check if catchment ID is present in the intersection DataFrame
        filename = os.path.splitext(os.path.basename(self.excel_file_path))[0]
        self.catchment_id = int(filename)
        self.intersection_df['Catchment ID'] = self.intersection_df['Catchment ID'].astype(int)
        filtered_df = self.intersection_df[self.intersection_df['Catchment ID'] == self.catchment_id]
        if not filtered_df.empty:
            self.Latitude = filtered_df['Latitude'].values[0]
            self.Longitude = filtered_df['Longitude'].values[0]
            self.resultsNon_Stationary_Stationary_Analysis_location_vary.append({
            'Catchment ID': self.catchment_id,
                'Latitude' : self.Latitude,
                'Longitude' : self.Longitude,
            'Stationary Location parameter': loc,
            'Stationary Shape parameter': shape,
            'Stationary Scale parameter': scale,
            **{f'Stationary Return value for {rp}-year': stationary_return_levels[rp].item() if isinstance(stationary_return_levels[rp], np.ndarray) else stationary_return_levels[rp] for rp in self.return_periods},
            'Location parameter(beta0)': beta0,
            f'Location parameter(beta1*(years-{self.start_year}))': beta1,
            f'Location parameter(beta2*(gmt))': beta2,
            f'Location parameter for year {self.end_year}': beta0 + beta1 * (self.end_year - self.start_year) + beta2 * (gmt_end.item() if isinstance(gmt_end, np.ndarray) else gmt_end),
            f'Shape parameter for year {self.end_year}': xi,
            f'Scale parameter for year {self.end_year}': sigma,
            **{f'Non-Stationary Return value for {rp}-year': non_stationary_return_levels_end_year[rp].item() if isinstance(non_stationary_return_levels_end_year[rp], np.ndarray) else non_stationary_return_levels_end_year[rp] for rp in self.return_periods},
            **{f'Change in Precipitation for return period {rp}-year': (non_stationary_return_levels_end_year[rp] - stationary_return_levels[rp]).item() if isinstance((non_stationary_return_levels_end_year[rp] - stationary_return_levels[rp]), np.ndarray) else non_stationary_return_levels_end_year[rp] - stationary_return_levels[rp] for rp in self.return_periods}  
            })
        else:
            self.Latitude = None
            self.Longitude = None
            print(f"No matching catchment ID {self.catchment_id} found in the intersection DataFrame. Skipping.")

        print(self.catchment_id)
        
       
#         print(f"Non-Stationary GEV Parameters: beta0={beta0}, beta1={beta1}, beta2={beta2}, sigma={sigma}, xi={xi}")
       
        # # Plot location parameter over the years
        # non_stationary_locations = beta0 + beta1 * (years - 1951) + beta2 * gmt
        # plt.figure(figsize=(10, 5))
        # plt.plot(years, non_stationary_locations, label='Location Parameter')
        # plt.xlabel('Year')
        # plt.ylabel('Location Parameter')
        # equation_text = f"Location = {beta0:.2f} + {beta1:.2f} * (Year - 1951) + {beta2:.2f} * GMT"
        # plt.text(0.05, 0.95, equation_text, transform=plt.gca().transAxes, fontsize=12, verticalalignment='top')
        # plt.legend()
        # plt.title('Location Parameter Over Time')
        # plt.show()
       
        # # Plot stationary and non-stationary return levels along with annual maximum precipitation
        # plt.figure(figsize=(10, 5))
        # for rp in self.return_periods:
        #     plt.plot(years, non_stationary_return_levels[rp], label=f'Non-Stationary {rp}-Year Return Level')
        # for rp in self.return_periods:
        #     plt.axhline(y=stationary_return_levels[rp], color='r', linestyle='--', label=f'Stationary {rp}-Year Return Level' if rp == self.return_periods[0] else "")
        # plt.plot(years, annual_max_precip, 'o-', label='Annual Maximum Precipitation', color='blue')
        # plt.xlabel('Year')
        # plt.ylabel('Return Level')
        # plt.legend()
        # plt.title('Comparison of Stationary and Non-Stationary Return Levels with Annual Maximum Precipitation')
        # plt.show()
       
        # # Plot stationary and non-stationary return levels
        # plt.figure(figsize=(10, 5))
        # for rp in self.return_periods:
        #     plt.plot(years, non_stationary_return_levels[rp], label=f'Non-Stationary {rp}-Year Return Level')
        # for rp in self.return_periods:
        #     plt.axhline(y=stationary_return_levels[rp], color='r', linestyle='--', label=f'Stationary {rp}-Year Return Level' if rp == self.return_periods[0] else "")
        # plt.xlabel('Year')
        # plt.ylabel('Return Level')
        # plt.legend()
        # plt.title('Comparison of Stationary and Non-Stationary Return Levels')
        # plt.show()
       
        # # Calculate Goodness of Fit
        # sample_quantiles = np.sort(annual_max_precip)
        # modeled_quantiles = gev.ppf((np.arange(len(annual_max_precip)) + 1) / (len(annual_max_precip) + 1), c=xi, loc=beta0 + beta1 * (years - 1951) + beta2 * gmt, scale=sigma)
        # plt.figure(figsize=(10, 5))
        # plt.plot(sample_quantiles, modeled_quantiles, 'o')
        # plt.plot([min(sample_quantiles), max(sample_quantiles)], [min(sample_quantiles), max(sample_quantiles)], 'r--')
        # plt.xlabel('Sample Quantiles')
        # plt.ylabel('Modeled Quantiles')
        # plt.title('QQ Plot of Non-Stationary GEV Model')
        # plt.show()
       

        # # Plot stationary GEV distribution
        # x = np.linspace(min(annual_max_precip), max(annual_max_precip), 100)
        # plt.plot(x, gev.pdf(x, c=shape, loc=loc, scale=scale), label='Stationary GEV', linestyle='-',color='black')

        # # Plot non-stationary GEV distribution for year 2014
        # mu_non_stationary = beta0 + beta1 * (2014 - 1951) + beta2 * gmt[-1]
        # plt.plot(x, gev.pdf(x, c=xi, loc=mu_non_stationary, scale=sigma), label=f'Non-Stationary GEV (Year 2014)', linestyle='-', color='g')
        # plt.xlabel('Precipitation (mm)')
        # plt.ylabel('Probability Density')
        # plt.title(f'GEV Distribution Comparison for {self.catchment_id}')
        # plt.legend()
        # plt.grid(True)
        # plt.tight_layout()
        # plt.show()

# Input

In [11]:
def process_model(model_name, data_start_year, start_year, end_year, return_periods,duration):
    # Set paths and parameters
    Model_Folder_Path = f"/DATA2/ALL_DATA_FOR_USERS/CMIP6_models_forcing/{model_name}"
    excel_file_path = r"/DATA1/ankit_new_data/CatchmentAreaPointsArea10000"
    intersection_point_path = r"/DATA1/ankit_new_data/Intersection Points After Preprocessing (Catchment area greater then 10000km2).xlsx"
    GMT_File_Path = f"/DATA1/ankit_new_data/gmt_data/Global_Mean_Temperature/{model_name}_Global_mean_temp.txt"

    # Initialize a list to collect results from all catchments
    all_results_location_vary = []
    all_results_location_window_size = []

    # Loop through each model folder in Model_Folder_Path
    model_folders = [f for f in os.listdir(Model_Folder_Path) if os.path.isdir(os.path.join(Model_Folder_Path, f))]
    for model_folder in model_folders:
        folder_path = os.path.join(Model_Folder_Path, model_folder)
        scenario = model_folder.split('_')[1]
        # print(scenario)
        # Loop through all Excel files in the excel_file_path
        excel_files = [file for file in os.listdir(excel_file_path) if file.endswith(".xlsx")]
        for i, excel_file in enumerate(excel_files):
            excel_path = os.path.join(excel_file_path, excel_file)
            analyzer = AnnualMaxPrecipitationAnalyzer(
                folder_path=folder_path, 
                excel_file_path=excel_path, 
                intersection_point_path =intersection_point_path, 
                GMT_File_Path=GMT_File_Path, 
                data_start_year=data_start_year, 
                start_year=start_year, 
                end_year=end_year, 
                return_periods=return_periods
            )
            analyzer.process_files(duration=duration)
            # analyzer.Non_Stationary_Stationary_Analysis_Using_Window_Size(window_size=25)
            analyzer.Non_Stationary_Stationary_Analysis_location_vary(scenario)
            # Extend the all_results list with the results from the current analyzer
            all_results_location_vary.extend(analyzer.resultsNon_Stationary_Stationary_Analysis_location_vary)
            print(f"Processed {i+1} catchments in folder {model_folder}")
        # window size
        # results_df_location_window_size = pd.DataFrame(all_results_location_window_size)
        # output_excel_path_splitwise = os.path.join(r"/DATA1/ankit_new_data", f"{model_folder}_windowsize.xlsx")
        # results_df_location_window_size.to_excel(output_excel_path_splitwise, index=False)
        # all_results_location_window_size.clear()
        # locationvary
        # Save the results to an Excel file named after the model folder
        results_df_location_vary = pd.DataFrame(all_results_location_vary)
        output_excel_path_locationvary = os.path.join(r"/DATA1/ankit_new_data/Different Model Analysis", f"{model_name}_{scenario}_Anlaysis_{start_year}-{end_year}_locationvary.xlsx")
        results_df_location_vary.to_excel(output_excel_path_locationvary, index=False)
        all_results_location_vary.clear()

In [ ]:
import os
import multiprocessing
# Directory path where the files are located
GMT_File_Path = "/DATA1/ankit_new_data/gmt_data/Global_Mean_Temperature"
# Function to extract model names from file names
def extract_model_name(filename):
    return filename.split('_')[0]  # Assuming model name is before '_Global_mean_temp.txt'
files = os.listdir(GMT_File_Path)
unique_models = set()
for file in files:
    if file.endswith('_Global_mean_temp.txt'):
        model_name = extract_model_name(file)
        unique_models.add(model_name)


# Function to be executed in parallel for each model
def process_model_wrapper(model_name):
    data_start_year = 1850
    start_year = 2071
    end_year = 2100
    return_periods = [25, 50, 100]
    duration = 1
    process_model(model_name, data_start_year, start_year, end_year, return_periods, duration)

if __name__ == "__main__":
    with multiprocessing.Pool() as pool:
        pool.map(process_model_wrapper, unique_models)
    print("All models processed.")
